In [1]:
import os
import io
import random
from flask import Flask, render_template, redirect, url_for, request, flash, Response
import sqlite3

## needed for dashboard to analysis part
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px ## pandas must be installed also
import pandas as pd #not needed ?
import numpy as np #not needed ?
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure


import base64 ## needed
import datetime ## needed
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta

In [2]:
US=None

conn = sqlite3.connect('IFRS16.db', check_same_thread=False)

c=conn.cursor()

c.execute("SELECT DISTINCT Category_Name, ID_Category FROM Categories")
categories=c.fetchall()

c.execute("SELECT DISTINCT Currency_Name, ID_Currency FROM Currencies")
currencies=c.fetchall()

c.execute("SELECT DISTINCT Company_Name, ID_Company FROM Companies")
companies=c.fetchall()

c.execute("SELECT DISTINCT Contract_Name, ID_Contract FROM Contracts")
names=c.fetchall()

c.execute("SELECT Contract_Name,Contract_Start_Date,Contract_End_Date,Contract_Monthly_Payment,User_Name,Category_Name, Company_Name, Currency_Name, IRR_Rate FROM Contracts INNER JOIN Categories ON Contracts.ID_Category=Categories.ID_Category INNER JOIN users ON Contracts.ID_Creator=Users.ID_User INNER JOIN Companies ON Contracts.ID_Company=Companies.ID_Company INNER JOIN Currencies ON Contracts.ID_Currency=Currencies.ID_Currency INNER JOIN IRR ON Currencies.ID_Currency=IRR.ID_Currency")
contracts=c.fetchall()

# function to count
def countOccurrences(arr, n, x): 
    res = 0
    for i in range(n): 
        if x == arr[i]: 
            res += 1
    return res 

format = '%Y-%m'

def convertToDate(date):
    for i in date:
        x=datetime.datetime.strptime(i, format).date()
        return x
    

In [3]:
template_dir = os.path.abspath('C:\\Users\\justy\\Desktop\\Test\\IFRS16')
static_dir = os.path.abspath('C:\\Users\\justy\\Desktop\\Test\\IFRS16\\css')
app = Flask(__name__,template_folder=template_dir, static_folder=static_dir)


@app.route('/', methods=["POST","GET"])
def default():
    if request.method == "POST":
        global US
        US=None
        message = "Logout successful"
        flash(message, 'info')
        return render_template ('home.html')
    
    return render_template ('home.html')

@app.route('/home', methods=["POST","GET"])
def home():
    if request.method == "POST":
        global US
        US=None
        message = "Logout successful"
        flash(message, 'info')
        return render_template ('home.html')
        
    return render_template ('home.html')

@app.route('/create_contract', methods=["POST","GET"])
def create_contract():
    c.execute("SELECT DISTINCT Category_Name, ID_Category FROM Categories")
    categories_up=c.fetchall()
    if request.method == "POST":
        name=request.form["contractName"]
        startDate=request.form["startDate"]
        endDate=request.form["endDate"]
        payment=request.form["payment"]
        category=request.form["dropdownCategory"]
        company=request.form["dropdownCompany"]
        currency=request.form["dropdownCurrency"]
    
        
        if float(payment)<=0:
            message = "Error!!! Monthly payment less than 0. Contract not created!"
            flash(message, 'error')
            return render_template ('create_contract.html',categories_up=categories_up, companies=companies, currencies=currencies)
        
        format = '%Y-%m'
        
        start = datetime.datetime.strptime(startDate, format).date()
        end = datetime.datetime.strptime(endDate, format).date()
        
        num_months=int((end.year - start.year)*12 + (end.month - start.month)+1)
        
        if num_months<=0:
            message = "Error!!! End date bigger than start date. Contract not created!"
            flash(message, 'error')
            return render_template ('create_contract.html',categories_up=categories_up, companies=companies, currencies=currencies)

        conval=float(payment)*num_months
        
        c.execute("SELECT IRR_Rate FROM IRR WHERE IRR.ID_Currency=?",(currency))
        IRR=c.fetchall()
        
        c.execute("SELECT Currency_Exchange_Rate FROM Currencies WHERE Currencies.ID_Currency=?",(currency))
        ExchangeRate=c.fetchall()
        ExchangeRate=str(ExchangeRate).replace(',', '')
        ExchangeRate=ExchangeRate.replace('[', '')
        ExchangeRate=ExchangeRate.replace(']', '')
        ExchangeRate=ExchangeRate.replace('(', '')
        ExchangeRate=ExchangeRate.replace(')', '')
        
        PV_sum_inner=np.empty(0)
        
        abc="'"
        len_mon=range(0,int(num_months),1)
        perc=str(IRR).replace('%', '')
        perc=perc.replace(',', '.',1)
        perc=perc.replace(',', '')
        perc=perc.replace('[', '')
        perc=perc.replace(']', '')
        perc=perc.replace('(', '')
        perc=perc.replace(')', '')
        perc=perc.replace(abc, '')
    
        disc=float(perc)/100
        disc_12=disc/12

        for z in len_mon:
            PV=1/pow((1+disc_12),z)*float(payment)
            PV_sum_inner=np.append(PV_sum_inner, PV)
            PV_x=round(sum(PV_sum_inner),2)
        
        contrInPLN=float(payment)*float(ExchangeRate)
        
        c.execute("SELECT ID_User FROM Users WHERE Users.User_Username=?",(US,))
        Usr=c.fetchall()
        Usr=str(Usr).replace(',', '')
        Usr=Usr.replace('[', '')
        Usr=Usr.replace(']', '')
        Usr=Usr.replace('(', '')
        Usr=Usr.replace(')', '')
        

        input_data=c.execute("INSERT INTO Contracts (Contract_Name, Contract_Start_Date, Contract_End_Date, Contract_Monthly_Payment,Contract_Monthly_Payment_PLN, ID_Creator,ID_Category, ID_Company, ID_Currency, Contract_Value, Contract_PV, Contract_No_Months) VALUES (?,?,?,?,?,?,?,?,?,?,?,?)",(name,startDate,endDate,payment,contrInPLN,Usr,category,company,currency,conval,PV_x,num_months))
        conn.commit()
        message = "Contract created"
        flash(message, 'info')
        return render_template ('create_contract.html',categories_up=categories_up, companies=companies, currencies=currencies)
    else:
        if US == None:
            message = "Please log in"
            flash(message, 'error')
            return render_template ('home.html')
        else:
            return render_template ('create_contract.html',categories_up=categories_up, companies=companies, currencies=currencies)

@app.route('/contract_overview', methods=["POST","GET"])
def contract_overview():
    
    c.execute("SELECT Contract_Name,Contract_Start_Date,Contract_End_Date,Contract_Monthly_Payment,User_Username,Category_Name, Company_Name, Currency_Name, IRR_Rate FROM Contracts INNER JOIN Categories ON Contracts.ID_Category=Categories.ID_Category INNER JOIN users ON Contracts.ID_Creator=Users.ID_User INNER JOIN Companies ON Contracts.ID_Company=Companies.ID_Company INNER JOIN Currencies ON Contracts.ID_Currency=Currencies.ID_Currency INNER JOIN IRR ON Currencies.ID_Currency=IRR.ID_Currency")
    data=c.fetchall()
    
    if request.method == "POST":
        dropdownName=request.form["selectName"]
        dropdownCategory=request.form["selectCategory"]
        dropdownCompany=request.form["selectCompany"]
        dropdownCurrency=request.form["selectCurrency"]
        dropdownPayment=request.form["selectPayment"]
        dropdownPayment2=request.form["selectPayment2"]
        
        if dropdownName == "allNames" and dropdownCategory =="allCategories" and dropdownCompany =="allCompanies" and dropdownCurrency =="allCurrencies" and dropdownPayment2 =="":
            c.execute("SELECT Contract_Name,Contract_Start_Date,Contract_End_Date,Contract_Monthly_Payment,User_Username,Category_Name, Company_Name, Currency_Name, IRR_Rate FROM Contracts INNER JOIN Categories ON Contracts.ID_Category=Categories.ID_Category INNER JOIN users ON Contracts.ID_Creator=Users.ID_User INNER JOIN Companies ON Contracts.ID_Company=Companies.ID_Company INNER JOIN Currencies ON Contracts.ID_Currency=Currencies.ID_Currency INNER JOIN IRR ON Currencies.ID_Currency=IRR.ID_Currency")
            data=c.fetchall()
            return render_template ('contract_overview.html', data=data, names=names, categories=categories, companies=companies, currencies=currencies)
        
        elif dropdownName != "allNames" and dropdownCategory =="allCategories" and dropdownCompany =="allCompanies" and dropdownCurrency =="allCurrencies" and dropdownPayment2 =="":
            c.execute("SELECT Contract_Name,Contract_Start_Date,Contract_End_Date,Contract_Monthly_Payment,User_Username,Category_Name, Company_Name, Currency_Name, IRR_Rate FROM Contracts INNER JOIN Categories ON Contracts.ID_Category=Categories.ID_Category INNER JOIN users ON Contracts.ID_Creator=Users.ID_User INNER JOIN Companies ON Contracts.ID_Company=Companies.ID_Company INNER JOIN Currencies ON Contracts.ID_Currency=Currencies.ID_Currency INNER JOIN IRR ON Currencies.ID_Currency=IRR.ID_Currency WHERE ID_Contract=?",(dropdownName,))
            data=c.fetchall()
            return render_template ('contract_overview.html', data=data, names=names, categories=categories, companies=companies, currencies=currencies)
        
        elif dropdownName == "allNames" and dropdownCategory !="allCategories" and dropdownCompany =="allCompanies" and dropdownCurrency =="allCurrencies" and dropdownPayment2 =="":
            c.execute("SELECT Contract_Name,Contract_Start_Date,Contract_End_Date,Contract_Monthly_Payment,User_Username,Category_Name, Company_Name, Currency_Name, IRR_Rate FROM Contracts INNER JOIN Categories ON Contracts.ID_Category=Categories.ID_Category INNER JOIN users ON Contracts.ID_Creator=Users.ID_User INNER JOIN Companies ON Contracts.ID_Company=Companies.ID_Company INNER JOIN Currencies ON Contracts.ID_Currency=Currencies.ID_Currency INNER JOIN IRR ON Currencies.ID_Currency=IRR.ID_Currency WHERE Contracts.ID_Category=?",(dropdownCategory,))
            data=c.fetchall()
            return render_template ('contract_overview.html', data=data, names=names, categories=categories, companies=companies, currencies=currencies)
        
        elif dropdownName == "allNames" and dropdownCategory =="allCategories" and dropdownCompany !="allCompanies" and dropdownCurrency =="allCurrencies" and dropdownPayment2 =="":
            c.execute("SELECT Contract_Name,Contract_Start_Date,Contract_End_Date,Contract_Monthly_Payment,User_Username,Category_Name, Company_Name, Currency_Name, IRR_Rate FROM Contracts INNER JOIN Categories ON Contracts.ID_Category=Categories.ID_Category INNER JOIN users ON Contracts.ID_Creator=Users.ID_User INNER JOIN Companies ON Contracts.ID_Company=Companies.ID_Company INNER JOIN Currencies ON Contracts.ID_Currency=Currencies.ID_Currency INNER JOIN IRR ON Currencies.ID_Currency=IRR.ID_Currency WHERE Contracts.ID_Company=?",(dropdownCompany,))
            data=c.fetchall()
            return render_template ('contract_overview.html', data=data, names=names, categories=categories, companies=companies, currencies=currencies)
        
        elif dropdownName == "allNames" and dropdownCategory =="allCategories" and dropdownCompany =="allCompanies" and dropdownCurrency !="allCurrencies" and dropdownPayment2 =="":
            c.execute("SELECT Contract_Name,Contract_Start_Date,Contract_End_Date,Contract_Monthly_Payment,User_Username,Category_Name, Company_Name, Currency_Name, IRR_Rate FROM Contracts INNER JOIN Categories ON Contracts.ID_Category=Categories.ID_Category INNER JOIN users ON Contracts.ID_Creator=Users.ID_User INNER JOIN Companies ON Contracts.ID_Company=Companies.ID_Company INNER JOIN Currencies ON Contracts.ID_Currency=Currencies.ID_Currency INNER JOIN IRR ON Currencies.ID_Currency=IRR.ID_Currency WHERE Contracts.ID_Currency=?",(dropdownCurrency,))
            data=c.fetchall()
            return render_template ('contract_overview.html', data=data, names=names, categories=categories, companies=companies, currencies=currencies)
        
        elif dropdownName == "allNames" and dropdownCategory =="allCategories" and dropdownCompany =="allCompanies" and dropdownCurrency =="allCurrencies" and dropdownPayment2 !="" and dropdownPayment =="less":
            c.execute("SELECT Contract_Name,Contract_Start_Date,Contract_End_Date,Contract_Monthly_Payment,User_Username,Category_Name, Company_Name, Currency_Name, IRR_Rate FROM Contracts INNER JOIN Categories ON Contracts.ID_Category=Categories.ID_Category INNER JOIN users ON Contracts.ID_Creator=Users.ID_User INNER JOIN Companies ON Contracts.ID_Company=Companies.ID_Company INNER JOIN Currencies ON Contracts.ID_Currency=Currencies.ID_Currency INNER JOIN IRR ON Currencies.ID_Currency=IRR.ID_Currency WHERE Contracts.Contract_Monthly_Payment < ?",(dropdownPayment2,))
            data=c.fetchall()
            return render_template ('contract_overview.html', data=data, names=names, categories=categories, companies=companies, currencies=currencies)
        
        elif dropdownName == "allNames" and dropdownCategory =="allCategories" and dropdownCompany =="allCompanies" and dropdownCurrency =="allCurrencies" and dropdownPayment2 !="" and dropdownPayment =="more":
            c.execute("SELECT Contract_Name,Contract_Start_Date,Contract_End_Date,Contract_Monthly_Payment,User_Username,Category_Name, Company_Name, Currency_Name, IRR_Rate FROM Contracts INNER JOIN Categories ON Contracts.ID_Category=Categories.ID_Category INNER JOIN users ON Contracts.ID_Creator=Users.ID_User INNER JOIN Companies ON Contracts.ID_Company=Companies.ID_Company INNER JOIN Currencies ON Contracts.ID_Currency=Currencies.ID_Currency INNER JOIN IRR ON Currencies.ID_Currency=IRR.ID_Currency WHERE Contracts.Contract_Monthly_Payment > ?",(dropdownPayment2,))
            data=c.fetchall()
            return render_template ('contract_overview.html', data=data, names=names, categories=categories, companies=companies, currencies=currencies)
        else:
            return render_template ('contract_overview.html', data=data, names=names, categories=categories, companies=companies, currencies=currencies)
        
        
    else:
        c.execute("SELECT Contract_Name,Contract_Start_Date,Contract_End_Date,Contract_Monthly_Payment,User_Username,Category_Name, Company_Name, Currency_Name, IRR_Rate FROM Contracts INNER JOIN Categories ON Contracts.ID_Category=Categories.ID_Category INNER JOIN users ON Contracts.ID_Creator=Users.ID_User INNER JOIN Companies ON Contracts.ID_Company=Companies.ID_Company INNER JOIN Currencies ON Contracts.ID_Currency=Currencies.ID_Currency INNER JOIN IRR ON Currencies.ID_Currency=IRR.ID_Currency")
        data=c.fetchall()
        if US == None:
            message = "Please log in"
            flash(message, 'error')
            return render_template ('home.html')
        else:
            return render_template ('contract_overview.html', data=data, names=names, categories=categories, companies=companies, currencies=currencies)

@app.route('/admin_panel', methods=["POST","GET"])
def admin_panel():
    if request.method == "POST":
        username_ad=request.form["username"]
        password_ad=request.form["password"]
        name_ad=request.form["Nam"]
        surname_ad=request.form["Sur"]
        job_ad=request.form["Job"]

        ## adding user
       
        c.execute("INSERT INTO Users (User_Username, User_Password, User_Name, User_Surname, User_Job_Type) VALUES (?,?,?,?,?)",(username_ad,password_ad,name_ad,surname_ad,job_ad))
        conn.commit()
        message = "User added successfully"
        flash(message, 'info')
        return render_template ('Admin_panel.html')
   
    
    if US == None:
        message = "Please log in"
        flash(message, 'error')
        return render_template ('home2.html')
   
    c.execute("SELECT User_Job_Type FROM Users WHERE Users.User_Username=?",(US,))
    Adm=c.fetchall()
    abc="'"
    Adm=str(Adm).replace(',', '')
    Adm=Adm.replace('[', '')
    Adm=Adm.replace(']', '')
    Adm=Adm.replace('(', '')
    Adm=Adm.replace(')', '')
    Adm=Adm.replace(abc, '')
        
    if Adm == 'admin':
        return render_template ('Admin_panel.html')
    else:
        message = "Olny admins has acces to this panel"
        flash(message, 'error')
        return render_template ('home.html')
    
    
@app.route('/add_user', methods=["POST","GET"])
def add_user():
    if request.method == "POST":
        username_ad=request.form["username"]
        password_ad=request.form["password"]
        name_ad=request.form["Nam"]
        surname_ad=request.form["Sur"]
        job_ad=request.form["Job"]

        ## adding user
       
        c.execute("INSERT INTO Users (User_Username, User_Password, User_Name, User_Surname, User_Job_Type) VALUES (?,?,?,?,?)",(username_ad,password_ad,name_ad,surname_ad,job_ad))
        conn.commit()
        message = "User added successfully"
        flash(message, 'info')
        return render_template ('Admin_panel.html')
   
    
    if US == None:
        message = "Please log in"
        flash(message, 'error')
        return render_template ('home.html')
   
    c.execute("SELECT User_Job_Type FROM Users WHERE Users.User_Username=?",(US,))
    Adm=c.fetchall()
    abc="'"
    Adm=str(Adm).replace(',', '')
    Adm=Adm.replace('[', '')
    Adm=Adm.replace(']', '')
    Adm=Adm.replace('(', '')
    Adm=Adm.replace(')', '')
    Adm=Adm.replace(abc, '')
        
    if Adm == 'admin':
        return render_template ('Add_User.html')
    else:
        message = "Olny admins has acces to this panel"
        flash(message, 'error')
        return render_template ('home.html')
    
    
@app.route('/add_cat', methods=["POST","GET"])
def add_cat():
    if request.method == "POST":
        cat_ad=request.form["cat_ad"]

        ## adding user
       
        c.execute("INSERT INTO Categories (Category_Name) VALUES (?)",(cat_ad,))
        conn.commit()
        message = "Category added successfully"
        flash(message, 'info')
        return render_template ('Admin_panel.html')
   
    
    if US == None:
        message = "Please log in"
        flash(message, 'error')
        return render_template ('home.html')
   
    c.execute("SELECT User_Job_Type FROM Users WHERE Users.User_Username=?",(US,))
    Adm=c.fetchall()
    abc="'"
    Adm=str(Adm).replace(',', '')
    Adm=Adm.replace('[', '')
    Adm=Adm.replace(']', '')
    Adm=Adm.replace('(', '')
    Adm=Adm.replace(')', '')
    Adm=Adm.replace(abc, '')
        
    if Adm == 'admin':
        return render_template ('Add_Cat.html')
    else:
        message = "Olny admins has acces to this panel"
        flash(message, 'error')
        return render_template ('home.html')
    

@app.route('/analyses', methods=["POST","GET"])
def analyses():
    #if request.method == "GET":
        c.execute("SELECT Contract_Name,Contract_Start_Date,Contract_End_Date,Contract_Monthly_Payment,User_Name,Contract_Value, Contract_PV, Contract_Monthly_Payment_PLN, Category_Name, Company_Name, Currency_Name, IRR_Rate FROM Contracts INNER JOIN Categories ON Contracts.ID_Category=Categories.ID_Category INNER JOIN users ON Contracts.ID_Creator=Users.ID_User INNER JOIN Companies ON Contracts.ID_Company=Companies.ID_Company INNER JOIN Currencies ON Contracts.ID_Currency=Currencies.ID_Currency INNER JOIN IRR ON Currencies.ID_Currency=IRR.ID_Currency")
        data=c.fetchall()
        
        c.execute("SELECT DISTINCT Category_Name, ID_Category FROM Categories")
        categories2=c.fetchall()
        
        ## pie

        z_sum=np.empty(0)
        a=np.array(data)
        b=np.array(categories2)
        
        df_cat=b[:,0]
        df_con=a[:,[8]]

        arr = df_con
        n = len(arr) 

        for t in df_cat:
            z=countOccurrences(arr,n,t)
            z_sum=np.append(z_sum,z)

        img = io.BytesIO()

        plt.pie(z_sum, labels=df_cat)

        plt.savefig(img, format='png')
        plt.close()
        img.seek(0)
        plot_url = base64.b64encode(img.getvalue()).decode('utf8')

        ## plot payment over time
        
        start_date=a[:,[1]]
        end_date=a[:,[2]]
        
        num_months_sum=np.empty(0)
        dates_sum=np.empty(0)
        val_sum=np.empty(0)
        ab=0
        for i,j in zip(start_date, end_date):
            val=a[:,[7]][ab]
            start = convertToDate(i)
            end=convertToDate(j)
            num_months=(end.year - start.year)*12 + (end.month - start.month)+1
            num_months_sum= np.append(num_months_sum, num_months)
            for i in range(num_months):
                dates = start + relativedelta(months=i)
                dates_sum=np.append(dates_sum,dates)
                val_sum=np.append(val_sum,val)
            ab=ab+1

        d = {'dates': dates_sum, 'col2': val_sum}

        min_date=min(d["dates"])
        max_date=max(d["dates"])

        tot_months=(max_date.year - min_date.year)*12 + (max_date.month - min_date.month)
        tot_months_2=range(0,tot_months+1,1)

        dat_sum=np.empty
        for i in tot_months_2:
            dat=min_date+pd.DateOffset(months=i)
            dat_sum=np.append(dat_sum,dat)

        dat_dat=pd.to_datetime(dat_sum[1:], format="%m/%d/%Y")

        sum_conv_all=np.empty
        for i in dat_sum:
            xxx=np.where(d["dates"] ==  i, d['col2'], 0)
            convertedArray = xxx.astype(float)
            sum_conv=sum(convertedArray)
            sum_conv_all=np.append(sum_conv_all,sum_conv)

        dat_val=sum_conv_all[2:,].tolist()
    
        img2 = io.BytesIO()

        plt.plot(dat_dat,dat_val)
        plt.tight_layout()
        
        plt.savefig(img2, format='png')
        plt.close()
        img2.seek(0)
        plot_url2 = base64.b64encode(img2.getbuffer()).decode('utf8')
        
        if US == None:
            message = "Please log in"
            flash(message, 'error')
            return render_template ('home.html')
        else:
            return render_template('analysis_piechart.html',plot_url=plot_url, plot_url2=plot_url2, data=data)

#@app.route('/sign_up')
#def sign_up():
#    return render_template ('create_contract.html')

@app.route('/login', methods=["POST","GET"])
def login():
    if request.method == "POST":
        username_temp=request.form["username"]
        password_temp=request.form["password"]
        
        c.execute("SELECT User_Username FROM Users")
        Users=c.fetchall()
        
        Login_correct_sum=[]
        
        for i in Users:
            abc="'"
            i=str(i).replace(',', '')
            i=i.replace('[', '')
            i=i.replace(']', '')
            i=i.replace('(', '')
            i=i.replace(')', '')
            i=i.replace(abc, '')
            
            if i == username_temp:
                Login_correct=1
                Login_correct_sum.append(Login_correct)
            else: 
                Login_correct=0
                Login_correct_sum.append(Login_correct)
                
        login_verify=sum(Login_correct_sum)
        if login_verify == 1:
            c.execute("SELECT User_Password FROM Users WHERE Users.User_Username=?",(username_temp,))
            password=c.fetchall()
            
            password=str(password).replace(',', '')
            password=password.replace('[', '')
            password=password.replace(']', '')
            password=password.replace('(', '')
            password=password.replace(')', '')
            password=password.replace(abc, '')
            
            if password==password_temp:
                global US
                US=username_temp
                message = "Login succesfull."
                flash(message, 'info')
                return render_template ('home.html')
                
            else:
                message = "Login failed. Please provide correct password."
                flash(message, 'error')
                return render_template ('Login.html')
                
        else: 
            message = "Login failed. Please provide correct username."
            flash(message, 'error')
            return render_template ('Login.html')

    return render_template ('Login.html')

@app.route('/contact', methods=["POST","GET"])
def contact():
    return render_template ('Contact.html')
    
@app.route('/logout')
def logout():
    logout_user()
    return redirect(url_for('home'))

In [ ]:
if __name__ == "__main__":
    app.secret_key = b'_5#y2L"F4Q8z\n\xec]/'
    app.config['TEMPLATES_AUTO_RELOAD'] = True
    app.debug=False
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Dec/2023 12:37:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2023 12:37:09] "GET /css/reset.css HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2023 12:37:09] "GET /css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2023 12:37:09] "GET /images/seperator.png HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2023 12:37:09] "GET /images/wwd_icon_1.png HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2023 12:37:09] "GET /images/wwd_icon_2.png HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2023 12:37:09] "GET /images/wwd_icon_3.png HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2023 12:37:09] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2023 12:37:49] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2023 12:37:49] "GET /css/reset.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Dec/2023 12:37:49] "GET /css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Dec/2023 12:37:51] "GET /create_contract HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2023 12:37:51] "GET /css/reset.cs

127.0.0.1 - - [11/Dec/2023 12:42:06] "GET /analyses HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2023 12:42:06] "GET /css/reset.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Dec/2023 12:42:06] "GET /css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Dec/2023 12:42:06] "GET /css/responsive.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Dec/2023 12:42:06] "GET /css/lists.css HTTP/1.1" 304 -
